In [24]:
import json
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "browser"  # Opens the plot in a web browser



In [15]:
file_path = '/home/yyj/Desktop/yyj/thesis/code/PETSQL/data/process/PPL_DEV.JSON-9_SHOT_Euclidean_mask_1034_1/results.json'

In [18]:

# 讀取 JSON 檔案
with open(file_path, "r", encoding="utf-8") as f:
    data = json.load(f)

In [25]:
# 初始化 DataFrame
records = []

for entry in data:
    index = entry["index"]
    weights = entry["current_weights"]
    for model, weight in weights.items():
        records.append({"index": index, "model": model, "weight": weight})

df = pd.DataFrame(records)

# 確保數據格式正確
df_pivot = df.pivot(index="index", columns="model", values="weight").reset_index()

# 這裡修改 value_name，避免與原始欄位名稱衝突
df_melted = df.pivot(index="index", columns="model", values="weight").reset_index().melt(
    id_vars=["index"], var_name="model", value_name="weight_value"
)

# 使用 Plotly 繪製可互動的折線圖
fig = px.line(
    df_melted,
    x="index",
    y="weight_value",
    color="model",
    markers=True,
    title="Dynamic Trend of Model Weights"
)

fig.update_layout(
    xaxis_title="Index",
    yaxis_title="Weight",
    hovermode="x unified",
    template="plotly_dark"
)

# 顯示互動式圖表
fig.show()

In [22]:
!pip install --upgrade nbformat
